<img src=picture\pic_2.jpg>


Powered by DALL·E 3

In [1]:
import pickle

import category_encoders as ce
import numpy as np
import optuna
import pandas as pd
pd.options.mode.copy_on_write = True
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

import plotly.express as px
import tqdm as notebook_tqdm
import xgboost as xg
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import (BaggingRegressor, ExtraTreesRegressor,
                              GradientBoostingRegressor,
                              HistGradientBoostingRegressor,
                              RandomForestRegressor)
from sklearn.feature_selection import (RFECV, SelectKBest,
                                       mutual_info_regression)
from sklearn.linear_model import ARDRegression, Lasso, LinearRegression, Ridge
from sklearn.metrics import (mean_absolute_error,
                             mean_absolute_percentage_error, r2_score)
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor



In [2]:
n_jobs = -1
random_state = 42

In [3]:
# # Производим десериализацию и извлекаем модель из файла формата pkl
with open('data/df.pkl', 'rb') as pkl_file:
    df = pickle.load(pkl_file)

df.head()

,baths,sqft,beds,target,city_is_capital,city_is_million,city_is_big,city_is_large,claster,distance_capital,distance_million,firepl,1_level,2_level,3_level,4_level,multi_level,multi_family,condo,Farms/Ranches,Townhouse,Single Family Home,pool,auction,foreclosure,new,school_rating_mean,number_school,school_distance_mean,Year_built,lotsize,Remodeled,forced_air_heating,gas_heating,electric_heating,pump_heating,central_heating,heat_bool,air_conditioner,ceiling_fan,cool_bool,Garage,Carport,Parking_lot,PublicSchoolOverallRanking,HDI (2021),Crime Rate
0,3.5,2900.0,4.0,418000.0,False,False,False,False,197.0,100.137884,655.199337,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,5.2,8,5.500000,0.0,2900.0,False,False,False,False,True,True,True,True,False,False,False,False,False,43.0,0.907,4.872
1,3.0,1947.0,3.0,310000.0,False,False,False,True,29.0,436.589228,1519.395159,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,4.0,3,1.299805,0.0,1947.0,False,False,False,False,False,False,False,False,False,False,False,False,False,13.0,0.940,5.759
2,NaN,897.0,2.0,209000.0,False,True,False,False,47.0,149.770696,11.871086,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,NaN,0,NaN,99.0,897.0,False,True,False,False,False,True,True,False,False,True,False,False,False,9.0,0.923,3.678
3,NaN,1507.0,NaN,181500.0,False,False,False,False,67.0,380.395709,1373.702768,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,2.3,4,3.800781,13.0,1507.0,True,False,False,True,True,True,True,True,False,True,False,False,False,42.0,0.911,3.922
4,2.0,3588.0,3.0,244900.0,False,False,False,False,31.0,178.458605,472.139498,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,3.8,6,6.398438,49.0,3588.0,False,True,False,False,False,True,True,False,False,True,False,False,False,16.0,0.930,3.700


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284051 entries, 0 to 284050
Data columns (total 47 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   baths                       231268 non-null  float64
 1   sqft                        284051 non-null  float64
 2   beds                        229584 non-null  float64
 3   target                      284051 non-null  float64
 4   city_is_capital             284051 non-null  bool   
 5   city_is_million             284051 non-null  bool   
 6   city_is_big                 284051 non-null  bool   
 7   city_is_large               284051 non-null  bool   
 8   claster                     284029 non-null  float64
 9   distance_capital            284029 non-null  float64
 10  distance_million            284029 non-null  float64
 11  firepl                      284051 non-null  bool   
 12  1_level                     284051 non-null  bool   
 13  2_level       

In [5]:
sorted(df.columns,key=str.lower)

['1_level',
 '2_level',
 '3_level',
 '4_level',
 'air_conditioner',
 'auction',
 'baths',
 'beds',
 'Carport',
 'ceiling_fan',
 'central_heating',
 'city_is_big',
 'city_is_capital',
 'city_is_large',
 'city_is_million',
 'claster',
 'condo',
 'cool_bool',
 'Crime Rate',
 'distance_capital',
 'distance_million',
 'electric_heating',
 'Farms/Ranches',
 'firepl',
 'forced_air_heating',
 'foreclosure',
 'Garage',
 'gas_heating',
 'HDI (2021)',
 'heat_bool',
 'lotsize',
 'multi_family',
 'multi_level',
 'new',
 'number_school',
 'Parking_lot',
 'pool',
 'PublicSchoolOverallRanking',
 'pump_heating',
 'Remodeled',
 'school_distance_mean',
 'school_rating_mean',
 'Single Family Home',
 'sqft',
 'target',
 'Townhouse',
 'Year_built']

In [6]:
object_columns = [s for s in df.columns if df[s].dtypes == 'object']
df = df.drop(object_columns, axis = 1)

# df = df.drop([               ],axis=1).copy() 
df = df.dropna()
print(df.shape)

X = df.drop('target',axis=1)
y = df.target


# Надо обязательно разбить на train, test, valid

X_train, X_tmp, y_train, y_tmp = train_test_split(X, y, train_size=0.5, random_state=random_state)

X_val, X_test, y_val, y_test  = train_test_split(X_tmp, y_tmp, train_size=0.5, random_state=random_state)


print(X_train.shape)
print(X_test.shape)

X_train.head()

baseline = LinearRegression()
baseline.fit(X_train,y_train)


predict = baseline.predict(X_test)
print(mean_absolute_percentage_error(y_test,predict))
# print(trans_target.inverse_transform(np.array(mean_absolute_error(y_test,predict)).reshape(-1, 1)))
print(mean_absolute_error(y_test,predict))
print(r2_score(y_test,predict))

(199056, 47)
(99528, 46)
(49764, 46)
0.5305529267122149
134751.35666123792
0.46305056367707376


In [7]:
models = [
    LinearRegression(n_jobs=n_jobs,),
    Lasso(random_state=random_state,max_iter=10000),
    Ridge(random_state=random_state,solver='svd'),
    DecisionTreeRegressor(random_state=random_state,),
    RandomForestRegressor(random_state=random_state,n_jobs=n_jobs,),
    HistGradientBoostingRegressor(random_state=random_state),
    GradientBoostingRegressor(random_state=random_state,),
    ExtraTreesRegressor(random_state=random_state,n_jobs=n_jobs,),
    BaggingRegressor(random_state=random_state,n_jobs=n_jobs,),
    ARDRegression(),
    KNeighborsRegressor(n_jobs=n_jobs,),
    # SVR(kernel='poly'),# исключила, тк хуже baseline и ужасно медленный
    LGBMRegressor(random_state=random_state,force_col_wise=True,n_jobs=n_jobs,),
    xg.XGBRegressor(random_state=random_state,n_jobs=n_jobs,),
    CatBoostRegressor(random_seed=random_state,verbose=0,)
          ]
result_mape = []
result_mae_S = []
result_r2 = []

# display(X_train)

for model in models:
    print(model)
    model.fit(X_train,y_train)
    predict = model.predict(X_test)
    result_mape.append(mean_absolute_percentage_error(y_test,predict))
    # result_mae_S.append(trans_target.inverse_transform(np.array(mean_absolute_error(y_test,predict)).reshape(-1, 1)))
    result_mae_S.append(mean_absolute_error(y_test,predict))
    result_r2.append(r2_score(y_test,predict))


models_name = []
for i in range(len(models)):
    models_name.append(str(models[i]))


df_rez = pd.DataFrame(data=[models_name,result_mape,result_mae_S,result_r2]).T
df_rez.columns = ['model','mape','mae_$','R2']
df_rez

LinearRegression(n_jobs=-1)
Lasso(max_iter=10000, random_state=42)
Ridge(random_state=42, solver='svd')
DecisionTreeRegressor(random_state=42)
RandomForestRegressor(n_jobs=-1, random_state=42)
HistGradientBoostingRegressor(random_state=42)
GradientBoostingRegressor(random_state=42)
ExtraTreesRegressor(n_jobs=-1, random_state=42)
BaggingRegressor(n_jobs=-1, random_state=42)
ARDRegression()
KNeighborsRegressor(n_jobs=-1)
LGBMRegressor(force_col_wise=True, random_state=42)
XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None

,model,mape,mae_$,R2
0,LinearRegression(n_jobs=-1),0.530553,134751.356661,0.463051
1,"Lasso(max_iter=10000, random_state=42)",0.530552,134740.669399,0.463052
2,"Ridge(random_state=42, solver='svd')",0.532985,134645.675178,0.461664
3,DecisionTreeRegressor(random_state=42),0.302126,91367.240848,0.627175
4,"RandomForestRegressor(n_jobs=-1, random_state=42)",0.237423,66915.008666,0.820985
5,HistGradientBoostingRegressor(random_state=42),0.296726,82457.36166,0.761738
6,GradientBoostingRegressor(random_state=42),0.38697,101797.228261,0.659745
7,"ExtraTreesRegressor(n_jobs=-1, random_state=42)",0.238197,67608.434081,0.813972
8,"BaggingRegressor(n_jobs=-1, random_state=42)",0.251465,71804.292549,0.797385
9,ARDRegression(),0.530897,134771.376067,0.462945


Дополнительно обучала LightAutoML (см. "First_year_real_estate_edditional_part.ipynb" ) 

Одна из лучших моделей это RandomForestRegressor. Эту модель буду обучать и улучшать.

In [8]:
model = RandomForestRegressor(random_state=random_state,
                            n_jobs=n_jobs,)
model.fit(X_train,y_train)
# # Производим сериализацию и записываем результат в файл формата pkl
with open(r'web\app\models\model.pkl', 'wb') as output:
    pickle.dump(model, output)

predict = model.predict(X_test)
print('mean_absolute_percentage_error ',mean_absolute_percentage_error(y_test,predict))
print('mean_absolute_error ',mean_absolute_error(y_test,predict))
print('r2_score ',r2_score(y_test,predict))
# print('mean_absolute_error_$' ,trans_target.inverse_transform(np.array(mean_absolute_error(y_test,predict)).reshape(-1, 1)))


feat_importances = pd.DataFrame(model.feature_importances_, index=X_train.columns, columns=["Importance"]).sort_values('Importance',ascending=False)
px.bar(feat_importances.round(3),text_auto=True)

mean_absolute_percentage_error  0.23742284458939553
mean_absolute_error  66915.00866592067
r2_score  0.82098503836017


In [9]:
sorted(df.columns,key=str.lower)

['1_level',
 '2_level',
 '3_level',
 '4_level',
 'air_conditioner',
 'auction',
 'baths',
 'beds',
 'Carport',
 'ceiling_fan',
 'central_heating',
 'city_is_big',
 'city_is_capital',
 'city_is_large',
 'city_is_million',
 'claster',
 'condo',
 'cool_bool',
 'Crime Rate',
 'distance_capital',
 'distance_million',
 'electric_heating',
 'Farms/Ranches',
 'firepl',
 'forced_air_heating',
 'foreclosure',
 'Garage',
 'gas_heating',
 'HDI (2021)',
 'heat_bool',
 'lotsize',
 'multi_family',
 'multi_level',
 'new',
 'number_school',
 'Parking_lot',
 'pool',
 'PublicSchoolOverallRanking',
 'pump_heating',
 'Remodeled',
 'school_distance_mean',
 'school_rating_mean',
 'Single Family Home',
 'sqft',
 'target',
 'Townhouse',
 'Year_built']

In [10]:
# Select the top 5 features using Univariate Feature Selection (change k as needed)
k = 30
selector = SelectKBest(score_func=mutual_info_regression, k=k)
X_selected = selector.fit_transform(X, y)

# Get the indices of the selected features
selected_indices = np.argsort(selector.scores_)[::-1][:k]
selected_features = X.columns[selected_indices]

# Print the selected features
print("Selected Features:")
print(selected_features)

Selected Features:
Index(['distance_capital', 'distance_million', 'claster', 'lotsize', 'sqft',
       'Year_built', 'Crime Rate', 'PublicSchoolOverallRanking', 'baths',
       'HDI (2021)', 'school_rating_mean', 'beds', 'school_distance_mean',
       'cool_bool', 'heat_bool', 'central_heating', 'foreclosure',
       'number_school', 'Garage', 'Remodeled', 'Single Family Home',
       'forced_air_heating', 'firepl', 'city_is_million', 'condo',
       'multi_family', '3_level', 'pool', 'city_is_big', 'city_is_capital'],
      dtype='object')


In [11]:
X_train_skb, X_tmp_skb, y_train_skb, y_tmp_skb = train_test_split(df[selected_features], df.target, train_size=0.5, random_state=random_state)

X_val_skb, X_test_skb, y_val_skb, y_test_skb  = train_test_split(X_tmp_skb, y_tmp_skb, train_size=0.5, random_state=random_state)

model_skb = RandomForestRegressor(random_state=random_state,n_jobs=n_jobs,)
model_skb.fit(X_train_skb,y_train_skb)
# # Производим сериализацию и записываем результат в файл формата pkl
with open(r'web\app\models\model_skb.pkl', 'wb') as output:
    pickle.dump(model_skb, output)

predict = model_skb.predict(X_test_skb)
print('mean_absolute_percentage_error ',mean_absolute_percentage_error(y_test_skb,predict))
print('mean_absolute_error ',mean_absolute_error(y_test_skb,predict))
print('r2_score ',r2_score(y_test_skb,predict))
# print('mean_absolute_error_$' ,trans_target.inverse_transform(np.array(mean_absolute_error(y_test_skb,predict)).reshape(-1, 1)))

# feat_importances = pd.DataFrame(model.feature_importances_, index=X_train.columns, columns=["Importance"]).sort_values('Importance',ascending=False)
# px.bar(feat_importances.round(3),text_auto=True)

mean_absolute_percentage_error  0.23811054955313937
mean_absolute_error  66906.60650713391
r2_score  0.821085884531771


In [12]:
predict = model_skb.predict(X_val_skb)
print('mean_absolute_percentage_error ',mean_absolute_percentage_error(y_val_skb,predict))
print('mean_absolute_error ',mean_absolute_error(y_val_skb,predict))
print('r2_score ',r2_score(y_val_skb,predict))
# print('mean_absolute_error_$' ,trans_target.inverse_transform(np.array(mean_absolute_error(y_test_skb,predict)).reshape(-1, 1)))

mean_absolute_percentage_error  0.2390639871308121
mean_absolute_error  67162.7126776768
r2_score  0.817702242265252


In [13]:
X = df.drop('target',axis=1)
y = df.target

estimator = RandomForestRegressor(random_state=random_state,
                                  n_jobs=n_jobs,)
min_features_to_select = 50
selector_RFECV = RFECV(estimator=estimator,
                 min_features_to_select=min_features_to_select,
                 step=1,
                 cv=3)
selector_RFECV = selector_RFECV.fit(X, y)
print(selector_RFECV.support_)
print(selector_RFECV.ranking_)
print(selector_RFECV.get_feature_names_out )

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1]
<bound method SelectorMixin.get_feature_names_out of RFECV(cv=3, estimator=RandomForestRegressor(n_jobs=-1, random_state=42),
      min_features_to_select=50)>


In [14]:
# # Производим сериализацию и записываем результат в файл формата pkl
with open(r'web\app\models\selector_RFECV.pkl', 'wb') as output:
    pickle.dump(selector_RFECV, output)

In [15]:
# # Производим десериализацию и извлекаем модель из файла формата pkl
with open(r'web\app\models\selector_RFECV.pkl', 'rb') as pkl_file:
    selector_RFECV = pickle.load(pkl_file)

In [16]:
selector_RFECV

RFECV(cv=3, estimator=RandomForestRegressor(n_jobs=-1, random_state=42),
      min_features_to_select=50)

In [17]:
feature_names = selector_RFECV.get_feature_names_out()
feature_names

array(['baths', 'sqft', 'beds', 'city_is_capital', 'city_is_million',
       'city_is_big', 'city_is_large', 'claster', 'distance_capital',
       'distance_million', 'firepl', '1_level', '2_level', '3_level',
       '4_level', 'multi_level', 'multi_family', 'condo', 'Farms/Ranches',
       'Townhouse', 'Single Family Home', 'pool', 'auction',
       'foreclosure', 'new', 'school_rating_mean', 'number_school',
       'school_distance_mean', 'Year_built', 'lotsize', 'Remodeled',
       'forced_air_heating', 'gas_heating', 'electric_heating',
       'pump_heating', 'central_heating', 'heat_bool', 'air_conditioner',
       'ceiling_fan', 'cool_bool', 'Garage', 'Carport', 'Parking_lot',
       'PublicSchoolOverallRanking', 'HDI (2021)', 'Crime Rate'],
      dtype=object)

In [18]:
sorted(feature_names,key=str.lower)

['1_level',
 '2_level',
 '3_level',
 '4_level',
 'air_conditioner',
 'auction',
 'baths',
 'beds',
 'Carport',
 'ceiling_fan',
 'central_heating',
 'city_is_big',
 'city_is_capital',
 'city_is_large',
 'city_is_million',
 'claster',
 'condo',
 'cool_bool',
 'Crime Rate',
 'distance_capital',
 'distance_million',
 'electric_heating',
 'Farms/Ranches',
 'firepl',
 'forced_air_heating',
 'foreclosure',
 'Garage',
 'gas_heating',
 'HDI (2021)',
 'heat_bool',
 'lotsize',
 'multi_family',
 'multi_level',
 'new',
 'number_school',
 'Parking_lot',
 'pool',
 'PublicSchoolOverallRanking',
 'pump_heating',
 'Remodeled',
 'school_distance_mean',
 'school_rating_mean',
 'Single Family Home',
 'sqft',
 'Townhouse',
 'Year_built']

In [19]:
X = df[feature_names]
y = df.target
X_train, X_tmp, y_train, y_tmp = train_test_split(X, y, train_size=0.5, random_state=random_state)

X_val, X_test, y_val, y_test  = train_test_split(X_tmp, y_tmp, train_size=0.5, random_state=random_state)

In [20]:
model_RFECV = RandomForestRegressor(random_state=random_state,n_jobs=n_jobs,)
model_RFECV.fit(X_train,y_train)
# # Производим сериализацию и записываем результат в файл формата pkl
with open(r'web\app\models\model_RFECV.pkl', 'wb') as output:
    pickle.dump(model_RFECV, output)

predict = model_RFECV.predict(X_test)
print('mean_absolute_percentage_error ',mean_absolute_percentage_error(y_test,predict))
print('mean_absolute_error ',mean_absolute_error(y_test,predict))
print('r2_score ',r2_score(y_test,predict))

# feat_importances = pd.DataFrame(model.feature_importances_, index=X_train.columns, columns=["Importance"]).sort_values('Importance',ascending=False)
# px.bar(feat_importances.round(3),text_auto=True)

mean_absolute_percentage_error  0.23742284458939553
mean_absolute_error  66915.00866592067
r2_score  0.82098503836017


In [21]:
predict = model_RFECV.predict(X_val)
print('mean_absolute_percentage_error ',mean_absolute_percentage_error(y_val,predict))
print('mean_absolute_error ',mean_absolute_error(y_val,predict))
print('r2_score ',r2_score(y_val,predict))

mean_absolute_percentage_error  0.23776400710122356
mean_absolute_error  67061.35058330509
r2_score  0.8179262861482987


In [29]:
def opt_Ext(trial):
    # задаем пространство поиска гиперпараметров

    n_estimators = trial.suggest_categorical('n_estimators',[20,50,100,200,300,500,700,1000])
    min_samples_leaf = trial.suggest_categorical('min_samples_leaf',[1,2,3,4,5,6,7,8,9,10,11,12])
    max_depth = trial.suggest_categorical('max_depth',[2,3,4,5,6,7,8,9,10,11,12,15,20,25,30,35,40,50])

    # создаем модель
    model = RandomForestRegressor(random_state=random_state,n_jobs=n_jobs,
                                n_estimators=n_estimators,
                                min_samples_leaf=min_samples_leaf,
                                max_depth=max_depth,
                                )

    model.fit(X_train,y_train)
    score = mean_absolute_percentage_error(y_val, model.predict(X_val))
    return score

In [31]:
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо минимизировать метрику direction="minimize"
stud = optuna.create_study(direction="minimize")


# ищем лучшую комбинацию гиперпараметров
stud.optimize(opt_Ext, n_trials=50)

[I 2024-09-11 21:31:37,718] A new study created in memory with name: no-name-8e0f427a-93ac-466b-a94c-815015083fee
[I 2024-09-11 21:33:05,219] Trial 0 finished with value: 0.25176934541174656 and parameters: {'n_estimators': 500, 'min_samples_leaf': 2, 'max_depth': 20}. Best is trial 0 with value: 0.25176934541174656.
[I 2024-09-11 21:33:50,785] Trial 1 finished with value: 0.4810111197406887 and parameters: {'n_estimators': 1000, 'min_samples_leaf': 4, 'max_depth': 6}. Best is trial 0 with value: 0.25176934541174656.
[I 2024-09-11 21:33:51,691] Trial 2 finished with value: 0.5127446529929911 and parameters: {'n_estimators': 20, 'min_samples_leaf': 2, 'max_depth': 5}. Best is trial 0 with value: 0.25176934541174656.
[I 2024-09-11 21:34:05,324] Trial 3 finished with value: 0.4808830241143102 and parameters: {'n_estimators': 300, 'min_samples_leaf': 8, 'max_depth': 6}. Best is trial 0 with value: 0.25176934541174656.
[I 2024-09-11 21:34:07,842] Trial 4 finished with value: 0.2983094043423

In [32]:
# # Производим сериализацию и записываем результат в файл формата pkl
with open(r'web\app\models\stud.pkl', 'wb') as output:
    pickle.dump(stud, output)

In [33]:
print(stud.best_params)

{'n_estimators': 200, 'min_samples_leaf': 1, 'max_depth': 50}


In [34]:
# # рассчитаем точность для тестовой выборки
best_model = RandomForestRegressor(random_state=random_state,n_jobs=n_jobs,
                                   n_estimators=stud.best_params['n_estimators'],
                                   min_samples_leaf=stud.best_params['min_samples_leaf'],
                                   max_depth=stud.best_params['max_depth'],
                                   )


best_model.fit(X_train,y_train)

# # Производим сериализацию и записываем результат в файл формата pkl
with open(r'web\app\models\best_model.pkl', 'wb') as output:
    pickle.dump(best_model, output)

In [35]:
predict = best_model.predict(X_test)
print('mean_absolute_percentage_error ',mean_absolute_percentage_error(y_test,predict))
print('mean_absolute_error ',mean_absolute_error(y_test,predict))
print('r2_score ',r2_score(y_test,predict))
# print('mean_absolute_error_$' ,trans_target.inverse_transform(np.array(mean_absolute_error(y_test,predict)).reshape(-1, 1)))


feat_importances = pd.DataFrame(best_model.feature_importances_, index=X_train.columns, columns=["Importance"]).sort_values('Importance',ascending=False)
px.bar(feat_importances.round(3),text_auto=True)

mean_absolute_percentage_error  0.23677637934675158
mean_absolute_error  66592.26597365996
r2_score  0.8226132589793766


In [36]:
def opt_Ext_cv(trial):
    # задаем пространство поиска гиперпараметров

    n_estimators = trial.suggest_categorical('n_estimators',[20,50,100,200,300,500,700,1000])
    min_samples_leaf = trial.suggest_categorical('min_samples_leaf',[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])
    max_depth = trial.suggest_categorical('max_depth',[2,3,4,5,6,7,8,9,10,11,12,15,20,25,30,35,40,50])

    # создаем модель
    model = RandomForestRegressor(random_state=random_state,n_jobs=n_jobs,
                                n_estimators=n_estimators,
                                min_samples_leaf=min_samples_leaf,
                                max_depth=max_depth,
                                )

    model.fit(X,y)

    score = -np.mean(cross_val_score(estimator=model, X=X, y=y,
                            scoring='neg_mean_absolute_percentage_error',
                            cv=5,
                            n_jobs=n_jobs))

    return score

In [37]:
# cоздаем объект исследования

stud_cv = optuna.create_study(direction="minimize")


# ищем лучшую комбинацию гиперпараметров
stud_cv.optimize(opt_Ext_cv, n_trials=50)

# # Производим сериализацию и записываем результат в файл формата pkl
with open(r'web\app\models\stud_cv.pkl', 'wb') as output:
    pickle.dump(stud_cv, output)

[I 2024-09-11 22:17:11,425] A new study created in memory with name: no-name-1512c532-ec1e-4642-83a8-31b5af93866f
[I 2024-09-11 22:19:59,255] Trial 0 finished with value: 0.2458069512239871 and parameters: {'n_estimators': 100, 'min_samples_leaf': 9, 'max_depth': 40}. Best is trial 0 with value: 0.2458069512239871.
[I 2024-09-11 22:44:47,594] Trial 1 finished with value: 0.21788616982679657 and parameters: {'n_estimators': 700, 'min_samples_leaf': 2, 'max_depth': 50}. Best is trial 1 with value: 0.21788616982679657.
[I 2024-09-11 22:45:22,401] Trial 2 finished with value: 0.23673216824417587 and parameters: {'n_estimators': 20, 'min_samples_leaf': 5, 'max_depth': 25}. Best is trial 1 with value: 0.21788616982679657.
[I 2024-09-11 22:48:55,312] Trial 3 finished with value: 0.4226073792261942 and parameters: {'n_estimators': 300, 'min_samples_leaf': 2, 'max_depth': 8}. Best is trial 1 with value: 0.21788616982679657.
[I 2024-09-11 23:00:09,933] Trial 4 finished with value: 0.260627497984

In [38]:
# # рассчитаем точность для тестовой выборки
best_model_cv = RandomForestRegressor(random_state=random_state,
                                    n_jobs=n_jobs,
                                    n_estimators=stud_cv.best_params['n_estimators'],
                                    min_samples_leaf=stud_cv.best_params['min_samples_leaf'],
                                    max_depth=stud_cv.best_params['max_depth'],
                                    )


best_model_cv.fit(X_train,y_train)

# # Производим сериализацию и записываем результат в файл формата pkl

# with open(r'web\app\models\best_model_cv.pkl', 'wb') as output:
#     pickle.dump(best_model_cv, output)

predict = best_model_cv.predict(X_test)
print('mean_absolute_percentage_error ',mean_absolute_percentage_error(y_test,predict))
print('mean_absolute_error ',mean_absolute_error(y_test,predict))
print('r2_score ',r2_score(y_test,predict))
# print('mean_absolute_error_$' ,trans_target.inverse_transform(np.array(mean_absolute_error(y_test,predict)).reshape(-1, 1)))


feat_importances = pd.DataFrame(best_model_cv.feature_importances_, index=X_train.columns, columns=["Importance"]).sort_values('Importance',ascending=False)
px.bar(feat_importances.round(3),text_auto=True)


mean_absolute_percentage_error  0.23711790276206343
mean_absolute_error  67049.50671872417
r2_score  0.8205364120404353
